In [3]:
%pip install pandas
%pip install xgboost
%pip install scikit-learn

  Using cached pandas-2.3.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.2-cp39-cp39-macosx_11_0_arm64.whl (10.8 MB)
Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl (5.3 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [34]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np

In [42]:
file_paths = "sampleData/lark.csv"

df = pd.read_csv(file_paths)
display(df.head())

,Order No,Order Date,Customer Name,Customer Contact,Customer Street,Customer Unit,Customer Postal Code,SKU,Item,Variant,Quantity,Platform Id,Delivery Date,Status,Custom Details
0,11875,21/8/25 18:53,San San Teo,6591080018,Blk 716 Woodlands Dr 70,08-122,730716,SC5,Vegas 1.2m Oak Shoe Cabinet,Assembly,1,NaN,NaN,NaN,NaN
1,11875,21/8/25 18:53,San San Teo,6591080018,Blk 716 Woodlands Dr 70,08-122,730716,CU,Soft Close Hinge Upgrade,Soft Close,1,NaN,NaN,NaN,NaN
2,11876,21/8/25 19:50,Jacob,6590626179,469 Ang Mo Kio Avenue 10,04-930,560469,SC5/WAL,Vegas 1.2m Walnut 4-Door Shoe Cabinet,Assembly,1,250821DUEB9P3Q,NaN,NaN,NaN
3,11877,22/8/25 20:04,Winnie Wong,6590412424,44 Sims Drive,12-175,380044,Elite/DivanSet/PU/Queen,Elite Divan Bed and Mattress Set,Faux Leather,1,NaN,NaN,NaN,NaN
4,11878,22/8/25 20:37,Kong Beng Liu,6596846842,307 Serangoon Avenue 2,14093,550307,SC5/WAL,Vegas 1.2m Walnut 4-Door Shoe Cabinet,Assembly,1,NaN,NaN,NaN,NaN


# XG Boost

In [43]:
# ensure order date is datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])

# aggregate demand weekly and by SKU
df = df.set_index('Order Date')
weekly = (
    df.groupby('SKU')['Quantity']
      .resample('W-MON')   # weekly, anchored on Mondays
      .sum()
      .reset_index()
)

# Sort by SKU and date
weekly = weekly.sort_values(['SKU', 'Order Date'])
display(weekly)

/var/folders/tk/ptln5ljx66g1yqhg_dd36_200000gn/T/ipykernel_34359/2313158920.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Order Date'] = pd.to_datetime(df['Order Date'])


,SKU,Order Date,Quantity
0,B16LD,2025-09-01,1
1,Basic/DivanSet/PU/Super Single,2025-08-25,1
2,CU,2025-08-25,8
3,CU,2025-09-01,5
4,DC1232/3075,2025-08-25,1
5,DC1289/3018WH,2025-09-01,1
6,DC1408/3045WH,2025-08-25,1
7,Elite/DivanSet/PU/Queen,2025-08-25,1
8,SB1048/3075,2025-08-25,1
9,SB1048/3075,2025-09-01,1


In [44]:
for lag in [1, 4, 12]:  # yesterday, last week
    weekly[f'lag_{lag}'] = weekly.groupby('SKU')['Quantity'].shift(lag)

In [45]:
display(weekly)

,SKU,Order Date,Quantity,lag_1,lag_4,lag_12
0,B16LD,2025-09-01,1,NaN,NaN,NaN
1,Basic/DivanSet/PU/Super Single,2025-08-25,1,NaN,NaN,NaN
2,CU,2025-08-25,8,NaN,NaN,NaN
3,CU,2025-09-01,5,8.0,NaN,NaN
4,DC1232/3075,2025-08-25,1,NaN,NaN,NaN
5,DC1289/3018WH,2025-09-01,1,NaN,NaN,NaN
6,DC1408/3045WH,2025-08-25,1,NaN,NaN,NaN
7,Elite/DivanSet/PU/Queen,2025-08-25,1,NaN,NaN,NaN
8,SB1048/3075,2025-08-25,1,NaN,NaN,NaN
9,SB1048/3075,2025-09-01,1,1.0,NaN,NaN


In [46]:
train = weekly[weekly['Order Date'] < '2025-08-29']
test = weekly[weekly['Order Date'] >= '2025-09-01']

X_train = train[['lag_1', 'lag_4']]
y_train = train['Quantity']
X_test = test[['lag_1', 'lag_4']]
y_test = test['Quantity']

# Train XGBoost
model = XGBRegressor(n_estimators=200, max_depth=5, learning_rate=0.1)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print("MAPE:", mape, "%")

MAPE: 64.28229528560972 %


# Temporal Fusion Transformer

In [48]:
%pip install pytorch-forecasting

2321.50s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached torch-2.8.0-cp39-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.2/825.2 kB 13.7 MB/s  0:00:00
Using cached fsspec-2025.7.0-py3-none-any.whl (199 kB)
Using cached PyYAML-6.0.2-cp39-cp39-macosx_11_0_arm64.whl (172 kB)
Using cached torch-2.8.0-cp39-none-macosx_11_0_arm64.whl 

In [ ]:
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.metrics import QuantileLoss
import pytorch_lightning as pl

In [ ]:
max_encoder_length = 12  # how many past weeks to look at
max_prediction_length = 4  # forecast 4 weeks ahead

In [51]:
weekly = weekly.sort_values(['Order Date'])
weekly['time_idx'] = (weekly['Order Date'].dt.year * 52 + 
                      weekly['Order Date'].dt.isocalendar().week).astype(int)

# Normalize so it starts at 0
weekly['time_idx'] = weekly['time_idx'] - weekly['time_idx'].min()

In [53]:
display(weekly)

,SKU,Order Date,Quantity,lag_1,lag_4,lag_12,time_idx
20,SC4,2025-08-25,2,NaN,NaN,NaN,0
6,DC1408/3045WH,2025-08-25,1,NaN,NaN,NaN,0
8,SB1048/3075,2025-08-25,1,NaN,NaN,NaN,0
7,Elite/DivanSet/PU/Queen,2025-08-25,1,NaN,NaN,NaN,0
28,SC5(X2)/WAL,2025-08-25,1,NaN,NaN,NaN,0
37,SS6202/3075,2025-08-25,1,NaN,NaN,NaN,0
11,SB6304/3064,2025-08-25,1,NaN,NaN,NaN,0
2,CU,2025-08-25,8,NaN,NaN,NaN,0
1,Basic/DivanSet/PU/Super Single,2025-08-25,1,NaN,NaN,NaN,0
4,DC1232/3075,2025-08-25,1,NaN,NaN,NaN,0


In [55]:
training = TimeSeriesDataSet(
    weekly,
    time_idx="time_idx",             
    target="Quantity",           
    group_ids=["SKU"],           
    static_categoricals=["SKU"],     
    time_varying_known_reals=["Order Date"],
    time_varying_unknown_reals=["Quantity"], 
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
)

# Dataloaders
train_dataloader = training.to_dataloader(train=True, batch_size=64)
val_dataloader = training.to_dataloader(train=False, batch_size=64)

# Define TFT model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.01,
    hidden_size=16,
    attention_head_size=4,
    dropout=0.1,
    loss=QuantileLoss(),
    log_interval=10,
)

# Train
trainer = pl.Trainer(max_epochs=20, gpus=1)
trainer.fit(tft, train_dataloader, val_dataloader)


/Users/joelsng/Documents/GitHub/levelsliving-IMS/.venv/lib/python3.9/site-packages/pytorch_forecasting/data/timeseries/_timeseries.py:1847: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 33 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__SKU': 'B16LD'}, {'__group_id__SKU': 'Basic/DivanSet/PU/Super Single'}, {'__group_id__SKU': 'CU'}, {'__group_id__SKU': 'DC1232/3075'}, {'__group_id__SKU': 'DC1289/3018WH'}, {'__group_id__SKU': 'DC1408/3045WH'}, {'__group_id__SKU': 'Elite/DivanSet/PU/Queen'}, {'__group_id__SKU': 'SB1048/3075'}, {'__group_id__SKU': 'SB1283/3075'}, {'__group_id__SKU': 'SB6304/3064'}]
  warnings.warn(


AssertionError: filters should not remove entries all entries - check encoder/decoder lengths and lags